https://www.tensorflow.org/guide/keras/train_and_evaluate/

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        layers.Dense(64, activation="relu", name="dense_1"),
        layers.Dense(64, activation="relu", name="dense_2"),
        layers.Dense(10, activation="softmax", name="predictions"),
    ]
)

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [4]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [5]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64, # tamanho do lote
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/2
782/782 [==============================] - 5s 3ms/step - loss: 0.3435 - sparse_categorical_accuracy: 0.9014 - val_loss: 0.1765 - val_sparse_categorical_accuracy: 0.9510
Epoch 2/2
782/782 [==============================] - 2s 3ms/step - loss: 0.1645 - sparse_categorical_accuracy: 0.9514 - val_loss: 0.1376 - val_sparse_categorical_accuracy: 0.9610


In [6]:
history.history

{'loss': [0.34345242381095886, 0.1644791215658188],
 'sparse_categorical_accuracy': [0.9013800024986267, 0.9513999819755554],
 'val_loss': [0.17648056149482727, 0.13763099908828735],
 'val_sparse_categorical_accuracy': [0.9509999752044678, 0.9610000252723694]}

In [7]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
79/79 [==============================] - 0s 2ms/step - loss: 0.1420 - sparse_categorical_accuracy: 0.9549
test loss, test acc: [0.1420472115278244, 0.9549000263214111]
Generate predictions for 3 samples
predictions shape: (3, 10)


In [10]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [11]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [12]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

In [13]:
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1)

625/625 [==============================] - 3s 4ms/step - loss: 0.3755 - sparse_categorical_accuracy: 0.8931 - val_loss: 0.2325 - val_sparse_categorical_accuracy: 0.9313


In [14]:
model = get_compiled_model()

# First, let's create a training Dataset instance.
# For the sake of our example, we'll use the same MNIST data as before.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

Epoch 1/3
782/782 [==============================] - 3s 3ms/step - loss: 0.3366 - sparse_categorical_accuracy: 0.9062
Epoch 2/3
782/782 [==============================] - 2s 3ms/step - loss: 0.1556 - sparse_categorical_accuracy: 0.9538
Epoch 3/3
782/782 [==============================] - 2s 3ms/step - loss: 0.1172 - sparse_categorical_accuracy: 0.9650
Evaluate
157/157 [==============================] - 1s 2ms/step - loss: 0.1201 - sparse_categorical_accuracy: 0.9642


{'loss': 0.12011219561100006,
 'sparse_categorical_accuracy': 0.9642000198364258}

In [15]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=1, validation_data=val_dataset)

782/782 [==============================] - 4s 4ms/step - loss: 0.3477 - sparse_categorical_accuracy: 0.9009 - val_loss: 0.1889 - val_sparse_categorical_accuracy: 0.9425


# Continua